In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output
import json
from datetime import datetime
from functools import partial

todo_items = []

# 공통 레이아웃
layout_full = widgets.Layout(width='100%')
layout_wide = widgets.Layout(width='750px')
layout_large = widgets.Layout(width='200px')
layout_medium = widgets.Layout(width='150px')
layout_small = widgets.Layout(width='100px')

# 진행률 바 + 라벨
overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='info', layout=layout_medium)
overall_progress_label = widgets.Label(value='(0%)', layout=layout_small)

# 설정 위젯
sort_dropdown = widgets.Dropdown(options=["마감일", "진행률", "중요도"], value="마감일", description="정렬:", layout=layout_medium)
hide_completed_toggle = widgets.ToggleButton(value=False, description='완료 항목 숨기기', icon='eye-slash', layout=layout_medium)
todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px', width='100%'))

# 전체 진행률 계산
def update_overall_progress():
    visible = [i for i in todo_items if not (hide_completed_toggle.value and i['progress'] == 100)]
    if not visible:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in visible) / len(visible)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"

# 리스트 갱신
def refresh_list(*args):
    todo_list_box.children = []
    headers = ['할 일', '진행률', '입력', '마감일', '중요도', '완료', '삭제']
    header_row = widgets.HBox([widgets.Label(h, layout=layout_medium) for h in headers],
                              layout=widgets.Layout(justify_content='center'))
    rows = [header_row]

    filtered = todo_items.copy()
    if hide_completed_toggle.value:
        filtered = [i for i in filtered if i['progress'] < 100]

    if sort_dropdown.value == '마감일':
        filtered.sort(key=lambda x: x['due'] or datetime.max)
    elif sort_dropdown.value == '진행률':
        filtered.sort(key=lambda x: x['progress'])
    elif sort_dropdown.value == '중요도':
        order = {'높음': 0, '보통': 1, '낮음': 2}
        filtered.sort(key=lambda x: order.get(x['priority'], 1))

    for item in filtered:
        title = widgets.Text(value=item['title'], layout=layout_medium, disabled=item['progress']==100)
        bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=layout_medium)
        slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_medium)
        due = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=layout_medium)
        priority = widgets.Label(value=item['priority'], layout=layout_medium)
        complete_btn = widgets.Button(description='완료', icon='check', button_style='success', layout=layout_medium)
        delete_btn = widgets.Button(description='삭제', icon='times', button_style='danger', layout=layout_medium)

        def on_change(change, item=item, bar=bar):
            item['progress'] = change['new']
            bar.value = change['new']
            update_overall_progress()
            refresh_list()

        def on_delete(b, item=item):
            todo_items.remove(item)
            update_overall_progress()
            refresh_list()

        def on_complete(b, item=item):
            item['progress'] = 100
            update_overall_progress()
            refresh_list()

        slider.observe(partial(on_change, item=item), names='value')
        complete_btn.on_click(partial(on_complete, item=item))
        delete_btn.on_click(partial(on_delete, item=item))

        row = widgets.HBox([title, bar, slider, due, priority, complete_btn, delete_btn],
                           layout=widgets.Layout(justify_content='center', margin='5px 0'))
        rows.append(row)

    todo_list_box.children = rows

# 입력 폼
title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_large)
slider_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_medium)
due_input = widgets.DatePicker(layout=layout_medium)
priority_input = widgets.Dropdown(options=["낮음", "보통", "높음"], value="보통", layout=layout_medium)
add_button = widgets.Button(description='추가', button_style='success', layout=layout_medium)

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': slider_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        slider_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_button.on_click(add_task)
sort_dropdown.observe(refresh_list, names='value')
hide_completed_toggle.observe(refresh_list, names='value')

# 저장 및 불러오기
def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([
            {
                **i,
                'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None
            } for i in todo_items
        ], f)

def load_data(b):
    try:
        with open('todo_data.json', 'r') as f:
            raw = json.load(f)
            todo_items.clear()
            for i in raw:
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                    'priority': i.get('priority', '보통')
                })
            update_overall_progress()
            refresh_list()
    except:
        print("불러오기 실패")

save_btn = widgets.Button(description='저장', button_style='info', layout=layout_medium)
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=layout_medium)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

# 전체 UI 출력
display(widgets.VBox([
    widgets.Label(value='DAVID mk10 - 모바일 최적화'),
    widgets.HBox([title_input], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([slider_input], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([due_input], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([priority_input, add_button], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([save_btn, load_btn], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([sort_dropdown, hide_completed_toggle], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([overall_progress_bar, overall_progress_label], layout=widgets.Layout(justify_content='center')),
    todo_list_box
], layout=layout_wide))

update_overall_progress()
refresh_list()
